In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os, os.path
import statistics

In [2]:
mid_scen = 'Mid Case'#######,'Mid Case',"Mid Case 95 by 2050","Mid Case 95 by 2035"]
mult_scen = "Mismatch_10Y"

In [3]:
mid_case_list = ['Mid Case','Mid Case 95 by 2050','Mid Case 95 by 2035']

orig_scen = ["Low_RE_Cost","Mid_Case","High_RE_Cost","Low_Trans","High_Trans",
             "Electrification_EnhancedFlex","Electrification","High_NG_Price","High_Demand_Growth","Low_Everything",
             "Low_NG_Price","Low_Nuclear_CCS_Cost","PTC_ITC_Ext","Reduced_RE_Resource",
            "Mid_Case_Base_Flex","Low_Demand_Growth"]
scen = orig_scen.copy()
for os in orig_scen:
    new1 = os+"_95_by_2050"
    new2 = os+"_95_by_2035"
    scen += [new1]
    scen += [new2]

analysis = ["v6.2jedi_mid","v6.2jedi_high","v6.2jedi_low","v6.2nei_mid","v6.2nei_low","v6.2nei_high"]
analysis_name = analysis

i = 0
for s in scen:
    for a in analysis:
        fpath = r"output/Dist_detailedjobzones_StdScen21_"+s+"_annual_state_"+a+".csv"
        job_data = pd.read_csv(fpath)
        job_data["Scenario"] = s.replace("_"," ")
        job_data["analysis"] = a
        if "2035" in s:
            job_data["ems_target"] = "95% by 2035"
        elif "2050" in s:
            job_data["ems_target"] = "95% by 2050"
        else:
            job_data["ems_target"] = "NNP"

        if i == 0:
            job_data_compiled = job_data.copy()
        else:
            job_data_compiled = job_data_compiled.append(job_data)
        i = i+1
job_data_compiled.head()

,index,2022,2050,Type,state,Retire,Remaining,Mismatch_1Y,Mismatch_5Y,Mismatch_10Y,MismatchMult_1y,MismatchMult_5y,MismatchMult_10y,Scenario,analysis,ems_target
0,Agriculture,0.539151,1.742939,1,AL,0.387831,0.151320,1.591619,1.500629,1.406960,2.952087,2.783321,2.609587,Low RE Cost,v6.2jedi_mid,NNP
1,Mining,0.000000,0.000000,1,AL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Low RE Cost,v6.2jedi_mid,NNP
2,Construction,0.000000,8.595279,1,AL,0.000000,0.000000,8.595279,20.863912,12.605718,inf,inf,inf,Low RE Cost,v6.2jedi_mid,NNP
3,Manufacturing,4.765846,6.362750,1,AL,3.285405,1.480441,4.882309,6.551173,5.446524,1.024437,1.374609,1.142824,Low RE Cost,v6.2jedi_mid,NNP
4,Fabricated Metals,0.000076,0.001255,1,AL,0.000053,0.000022,0.001233,0.001180,0.001131,16.294978,15.601380,14.950377,Low RE Cost,v6.2jedi_mid,NNP


### Sectoral Impact

In [4]:
count = 0
for _scen in scen:
    for _analysis in analysis:
        scen_name = _scen.replace("_"," ")
        job_data_all = job_data_compiled.query("Scenario==@scen_name").query("analysis==@_analysis")
        mid_name = job_data_all["ems_target"].iloc[0]
        job_data_jedi = job_data_all.copy()
        job_data_jedi = job_data_jedi.groupby(["state","index"]).sum()
        job_data_jedi = job_data_jedi.reset_index()
        job_data_jedi["MismatchMult"] = job_data_jedi[mult_scen]/job_data_jedi["2022"]
        job_data_jedi["MismatchMult"] = job_data_jedi["MismatchMult"].replace(np.nan,0)
        job_data_jedi_pivot = job_data_jedi.pivot(index="state",values="MismatchMult",columns="index")
        job_data_jedi_pivot = job_data_jedi_pivot.replace(np.inf,100000)
        job_data_jedi_pivot["emissions"] = mid_name
        job_data_jedi_pivot["scenario"] = _scen
        job_data_jedi_pivot["analysis"] = _analysis
        if count == 0:
            sector_data = job_data_jedi_pivot
        else:
            sector_data = sector_data.append(job_data_jedi_pivot)
        count = count + 1
sector_data.to_csv("Compiled_employment_change_sectoral.csv")
sector_data

index,Agriculture,Construction,Electrical Equipment,FIRE,Fabricated Metals,Government,Machinery,Manufacturing,Mining,Misc. Services,Professional Services,Retail Trade,TCPU,Wholesale Trade,emissions,scenario,analysis
state,,,,,,,,,,,,,,,,,
AL,2.609587,100000.000000,5.907113,0.685516,14.950377,2.259642,0.702829,1.142824,-0.036945,0.646620,0.656260,1.311880,0.579400,0.662616,NNP,Low_RE_Cost,v6.2jedi_mid
AR,2.161584,100000.000000,4.581668,1.002382,8.239400,10.561239,0.796376,1.551844,0.726650,1.228445,0.616397,2.857079,0.899900,1.421521,NNP,Low_RE_Cost,v6.2jedi_mid
AZ,0.271227,100000.000000,3.935847,0.786198,-0.049450,2.620680,0.724254,1.275274,2.866826,0.658854,0.639673,1.534352,1.124350,0.880072,NNP,Low_RE_Cost,v6.2jedi_mid
CA,0.008509,288.068294,2.175712,0.757720,-0.052237,2.385370,5.252385,2.237463,-0.055926,1.426517,0.515006,2.159094,1.241132,1.111142,NNP,Low_RE_Cost,v6.2jedi_mid
CO,1.048113,-0.261277,3.506358,0.795019,-0.244119,1.339299,0.210541,0.274336,0.065762,0.809279,0.483585,0.972799,0.882211,1.464002,NNP,Low_RE_Cost,v6.2jedi_mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VT,4.468294,100000.000000,1.228267,1.477419,1.384175,6.812450,-0.130022,3.258735,1.394927,5.182184,0.837610,5.877768,2.089147,3.417579,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
WA,283.932690,8.782356,1.261821,0.794129,3.225024,2.255849,-0.274987,1.023307,5.348335,0.979977,0.683452,1.512620,1.094226,0.931437,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
WI,0.358937,2153.226301,15.033613,0.816371,8.401826,3.359685,0.592685,1.806271,12.703896,0.787134,0.619417,1.490195,1.016530,2.374209,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high


### Sector Impact Overall

In [5]:
count = 0
for _scen in scen:
    for _analysis in analysis:
        scen_name = _scen.replace("_"," ")
        job_data_all = job_data_compiled.query("Scenario==@scen_name").query("analysis==@_analysis")
        mid_name = job_data_all["ems_target"].iloc[0]
        job_data_jedi = job_data_all.copy()
        job_data_jedi = job_data_jedi.groupby("index").sum()
        job_data_jedi = job_data_jedi.reset_index()
        job_data_jedi["MismatchMult"] = job_data_jedi[mult_scen]/job_data_jedi["2022"]
        job_data_jedi["MismatchMult"] = job_data_jedi["MismatchMult"].replace(np.nan,0)
        job_data_jedi = job_data_jedi[["index","MismatchMult"]]
        #job_data_jedi_pivot = job_data_jedi.pivot(index="state",values="MismatchMult",columns="Type")
        #job_data_jedi_pivot = job_data_jedi_pivot.replace(np.inf,100000)
        job_data_jedi["emissions"] = mid_name
        job_data_jedi["scenario"] = _scen
        job_data_jedi["analysis"] = _analysis
        if count == 0:
            total_sector_data = job_data_jedi
        else:
            total_sector_data = total_sector_data.append(job_data_jedi)
        count = count + 1
total_sector_data = total_sector_data.set_index("index")
total_sector_data.to_csv("Compiled_employment_change_sectoral_sum.csv")
total_sector_data       

,MismatchMult,emissions,scenario,analysis
index,,,,
Agriculture,0.345371,NNP,Low_RE_Cost,v6.2jedi_mid
Construction,2.588200,NNP,Low_RE_Cost,v6.2jedi_mid
Electrical Equipment,2.611026,NNP,Low_RE_Cost,v6.2jedi_mid
FIRE,0.794273,NNP,Low_RE_Cost,v6.2jedi_mid
Fabricated Metals,-0.137195,NNP,Low_RE_Cost,v6.2jedi_mid
...,...,...,...,...
Misc. Services,0.815148,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
Professional Services,0.448695,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
Retail Trade,1.330419,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high


### Skill Level Impact

In [6]:
count = 0
for _scen in scen:
    for _analysis in analysis:
        scen_name = _scen.replace("_"," ")
        job_data_all = job_data_compiled.query("Scenario==@scen_name").query("analysis==@_analysis")
        mid_name = job_data_all["ems_target"].iloc[0]
        job_data_jedi = job_data_all.copy()
        job_data_jedi = job_data_jedi.groupby(["state","Type"]).sum()
        job_data_jedi = job_data_jedi.reset_index()
        job_data_jedi["MismatchMult"] = job_data_jedi[mult_scen]/job_data_jedi["2022"]
        job_data_jedi["MismatchMult"] = job_data_jedi["MismatchMult"].replace(np.nan,0)
        job_data_jedi_pivot = job_data_jedi.pivot(index="state",values="MismatchMult",columns="Type")
        job_data_jedi_pivot = job_data_jedi_pivot.replace(np.inf,100000)
        job_data_jedi_pivot["emissions"] = mid_name
        job_data_jedi_pivot["scenario"] = _scen
        job_data_jedi_pivot["analysis"] = _analysis
        if count == 0:
            zone_data = job_data_jedi_pivot
        else:
            zone_data = zone_data.append(job_data_jedi_pivot)
        count = count + 1
zone_data.to_csv("Compiled_employment_change_jobzone.csv")
zone_data

Type,1,2,3,4,5,emissions,scenario,analysis
state,,,,,,,,
AL,1.347622,0.509689,0.774484,0.629470,0.524289,NNP,Low_RE_Cost,v6.2jedi_mid
AR,2.431044,1.245582,1.391552,0.918893,0.736931,NNP,Low_RE_Cost,v6.2jedi_mid
AZ,1.873082,1.237740,1.199898,0.889443,0.700864,NNP,Low_RE_Cost,v6.2jedi_mid
CA,1.684446,1.415953,1.397666,1.081018,0.732937,NNP,Low_RE_Cost,v6.2jedi_mid
CO,0.180605,0.643272,0.591262,0.739429,0.441324,NNP,Low_RE_Cost,v6.2jedi_mid
...,...,...,...,...,...,...,...,...
VT,5.441366,3.090040,2.758464,2.337636,2.370968,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
WA,3.071268,1.165739,1.073232,0.907528,0.786514,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
WI,1.543867,1.294716,1.542653,1.042154,0.750114,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high


### Skill Level Overall

In [7]:
count = 0
for _scen in scen:
    for _analysis in analysis:
        scen_name = _scen.replace("_"," ")
        job_data_all = job_data_compiled.query("Scenario==@scen_name").query("analysis==@_analysis")
        mid_name = job_data_all["ems_target"].iloc[0]
        job_data_jedi = job_data_all.copy()
        job_data_jedi = job_data_jedi.groupby("Type").sum()
        job_data_jedi = job_data_jedi.reset_index()
        job_data_jedi["MismatchMult"] = job_data_jedi[mult_scen]/job_data_jedi["2022"]
        job_data_jedi["MismatchMult"] = job_data_jedi["MismatchMult"].replace(np.nan,0)
        job_data_jedi = job_data_jedi[["Type","MismatchMult"]]
        #job_data_jedi_pivot = job_data_jedi.pivot(index="state",values="MismatchMult",columns="Type")
        #job_data_jedi_pivot = job_data_jedi_pivot.replace(np.inf,100000)
        job_data_jedi["emissions"] = mid_name
        job_data_jedi["scenario"] = _scen
        job_data_jedi["analysis"] = _analysis
        if count == 0:
            total_zone_data = job_data_jedi
        else:
            total_zone_data = total_zone_data.append(job_data_jedi)
        count = count + 1
total_zone_data = total_zone_data.set_index("Type")
total_zone_data.to_csv("Compiled_employment_change_jobzone_sum.csv")
total_zone_data                                              

,MismatchMult,emissions,scenario,analysis
Type,,,,
1,0.318002,NNP,Low_RE_Cost,v6.2jedi_mid
2,0.851285,NNP,Low_RE_Cost,v6.2jedi_mid
3,0.916803,NNP,Low_RE_Cost,v6.2jedi_mid
4,0.693965,NNP,Low_RE_Cost,v6.2jedi_mid
5,0.442798,NNP,Low_RE_Cost,v6.2jedi_mid
...,...,...,...,...
1,0.900449,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
2,1.342471,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high
3,2.058036,95% by 2035,Low_Demand_Growth_95_by_2035,v6.2nei_high


## Total Sum 

In [9]:
total_data = job_data_compiled.groupby(["state","Scenario","analysis","ems_target"]).sum()
total_data = total_data.reset_index()
total_data["MismatchMult"] = total_data[mult_scen]/total_data["2022"]
total_data["MismatchMult"] = total_data["MismatchMult"].replace(np.nan,0)
total_data = total_data[["state","Scenario","analysis","ems_target","MismatchMult"]]
total_data = total_data.set_index("state")
total_data = total_data.rename(columns={"Scenario":"scenario", "ems_target":"emissions"})
total_data.to_csv("Compiled_employment_change_sum.csv")
total_data

,scenario,analysis,emissions,MismatchMult
state,,,,
AL,Electrification,v6.2jedi_high,NNP,0.788139
AL,Electrification,v6.2jedi_low,NNP,0.810023
AL,Electrification,v6.2jedi_mid,NNP,0.791441
AL,Electrification,v6.2nei_high,NNP,0.758180
AL,Electrification,v6.2nei_low,NNP,0.776232
...,...,...,...,...
WY,Reduced RE Resource 95 by 2050,v6.2jedi_low,95% by 2050,-0.138098
WY,Reduced RE Resource 95 by 2050,v6.2jedi_mid,95% by 2050,-0.151636
WY,Reduced RE Resource 95 by 2050,v6.2nei_high,95% by 2050,-0.155912


In [15]:
scen_total_data = job_data_compiled.groupby(["Scenario","analysis","ems_target"]).sum()
scen_total_data = scen_total_data.reset_index()
scen_total_data["MismatchMult"] = scen_total_data[mult_scen]/scen_total_data["2022"]
scen_total_data["MismatchMult"] = scen_total_data["MismatchMult"].replace(np.nan,0)
scen_total_data = scen_total_data[["Scenario","analysis","ems_target","MismatchMult"]]
scen_total_data

,Scenario,analysis,ems_target,MismatchMult
0,Electrification,v6.2jedi_high,NNP,1.182991
1,Electrification,v6.2jedi_low,NNP,1.207995
2,Electrification,v6.2jedi_mid,NNP,1.183393
3,Electrification,v6.2nei_high,NNP,1.122087
4,Electrification,v6.2nei_low,NNP,1.146331
...,...,...,...,...
283,Reduced RE Resource 95 by 2050,v6.2jedi_low,95% by 2050,0.867916
284,Reduced RE Resource 95 by 2050,v6.2jedi_mid,95% by 2050,0.842828
285,Reduced RE Resource 95 by 2050,v6.2nei_high,95% by 2050,0.814841
286,Reduced RE Resource 95 by 2050,v6.2nei_low,95% by 2050,0.841648


In [21]:
scen_total_data.query("ems_target=='NNP'").sort_values(by="MismatchMult").iloc[[0,-1]]

,Scenario,analysis,ems_target,MismatchMult
255,PTC ITC Ext,v6.2nei_high,NNP,0.740845
1,Electrification,v6.2jedi_low,NNP,1.207995


In [24]:
scen_total_data.query("ems_target=='95% by 2050'").sort_values(by="MismatchMult").iloc[[0,-1]]

,Scenario,analysis,ems_target,MismatchMult
87,High RE Cost 95 by 2050,v6.2nei_high,95% by 2050,0.792181
13,Electrification 95 by 2050,v6.2jedi_low,95% by 2050,1.653191


In [25]:
scen_total_data.query("ems_target=='95% by 2035'").sort_values(by="MismatchMult").iloc[[0,-1]]

,Scenario,analysis,ems_target,MismatchMult
135,Low Everything 95 by 2035,v6.2nei_high,95% by 2035,1.245387
25,Electrification EnhancedFlex 95 by 2035,v6.2jedi_low,95% by 2035,2.154746
